In [2]:
#%pip install transformers==4.34.0
#pip install git+https://github.com/huggingface/transformers

In [4]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments
import torch

ImportError: huggingface-hub>=0.19.3,<1.0 is required for a normal functioning of this module, but found huggingface-hub==0.17.3.
Try: pip install transformers -U or pip install -e '.[dev]' if you're working with git main

In [3]:
MODEL_NAME = "Mistral-7B-Instruct-v0.1"

In [4]:
def load_peft_model(base_model: str, peft_model: str):
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        quantization_config=bnb_config,
        #device_map="auto",
        device_map="cuda:0",
        # trust_remote_code=True,
        local_files_only=True,
        # use_safetensors=True
    )
    if peft_model is not None:
        if os.path.exists(f"{peft_model}/adapter_config.json"):
            print(f"Loading PEFT {peft_model}")
            model.load_adapter(peft_model)
        else:
            print("WARNING: PEFT_MODEL NOT EXISTS!!!")
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer


In [5]:
def create_llama2_generation_prompt(system_message, question: str):
    if system_message is not None:
        return ("<s>[INST] <<SYS>>\n{system_message}\n<</SYS>>\n\n{user_input} [/INST]"
                .format(system_message=system_message, user_input=question))
    prompt_template = """<s>[INST] {user_input} [/INST]"""
    return prompt_template.format(user_input=question)


def ask_llama2_instruction_prompt(model, generation_config, tokenizer, device, question: str):
    system_msg = (
        "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. "
        "Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. "
        "Please ensure that your responses are socially unbiased and positive in nature.\n"
        "If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. "
        "If you don't know the answer to a question, please don't share false information.")
    prompt = create_llama2_generation_prompt(system_msg, question)
    encoding = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config
    )

    resp = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return resp.replace(prompt, "")

In [6]:
class LLM:
    def __init__(self):
        model, tokenizer, generation_config = self.load_model()
        self.model = model
        self.tokenizer = tokenizer
        self.generation_config = generation_config
        self.device = 'cuda'

    def load_model(self):
        base_model = f"../models/{MODEL_NAME}"
        model, tokenizer = load_peft_model(base_model, None)
        generation_config = model.generation_config
        generation_config.max_new_tokens = 1024
        generation_config.temperature = 0.2
        generation_config.do_sample = True
        generation_config.top_p = 0.9
        generation_config.num_return_sequences = 1
        generation_config.pad_token_id = tokenizer.eos_token_id
        generation_config.eos_token_id = tokenizer.eos_token_id
        return model, tokenizer, generation_config

    def ask(self, user_input: str):
        answer = ask_llama2_instruction_prompt(model=self.model,
                                               generation_config=self.generation_config,
                                               tokenizer=self.tokenizer,
                                               device=self.device,
                                               question=user_input)
        return answer


In [7]:
PROMPT = """{content}
----------
According to the above content, please generate several questions,
the answer of which must contain all or part of the content above.
The total content of the generated questions and answers must cover at least 80% of the above content.
Please do not generate duplicate questions.
Every question number must be prefixed with the keyword "Question".
When generating questions, do not give me the question number.
Response example:
   Question: How to play
   Answer: You muse stardy
   Question: Hi
   Answer: Hello """

In [8]:
content = """Introduction to Live Dealer Dragon Tiger
The objective of the game is to bet on whichever of the two hands, the Dragon's hand or the Tiger's hand that the Player thinks will have the highest card value. The Player can also bet on Tie. Dealer will draw one card for the Dragon's hand and one card for the Tiger's hand. No additional cards will be drawn for each hand.

The theoretical return to player of this game is 96.26%
Over a long period of time, the game is likely to average a return to the Player of 96.265% of the total bets made.

Side Bets

Live Dragon Tiger game also has side bets: Odd/Even, Big/Small, Black/Red.
The side bets are the minor bets that can be placed by the Player.
NOTE: When betting on the side bets, the Player loses all his/her bets on the side bets he/she placed if the card is dealt as 7 . Over the 50th round of each time, it is not acceptable to bet with side bet.

Bet Options and Payout:
Dragon: Bet on Dragon's hand. 50% of the bet will be returned to Player if the result is Tie. 1:1.
Tiger: Bet on Tiger's hand. 50% of the bet will be returned to Player if the result is Tie. 1:1.
Tie: Bet on Tie which the value of both Dragon and Tiger's hand are equal/same hand rank value. 8:1.
Odd: Bet on value of Dragon's hand (left) or Tiger's hand (right) will be an Odd number. Bet loses if the result is 7. 1:1.
Even: Bet on value of Dragon's hand (left) or Tiger's hand (right) will be an Even number. Bet loses if the result is 7. 1:1.
Small: Bet on value of Dragon's hand (left) or Tiger's hand (right) will be a Small number, including Ace to 6. Bet loses if the result is 7. 1:1.
Big: Bet on value of Dragon's hand (left) or Tiger's hand (right) will be a Big number, including 8 to K. Bet loses if the result is 7. 1:1.
Red: Bet on poker colour of Dragon hand's (left) or Tiger hand's (right) will be Red. Bet loses if the result is 7. 1:1.
Black: Bet on poker colour Dragon hand's (left) or Tiger hand's (right) will be Black. Bet loses if the result is 7. 1:1.

Card Values
Cards in Live Dragon Tiger are ranked from the lowest to the highest starting from Ace to King.
Aces are counted as one
Jacks are counted as 11
Queens are counted as 12
Kings are counted as 13

Burn Card
The dealer will burn one card in the beginning of every round in the games.

Reshuffles
The Operator reserves the rights to reshuffle the particular decks of cards if any human or non-human errors occur during cards shuffling.

Electronic Malfunctions
The card is scanned by the dealer, and the result will be displayed on the Player's screen. If any card fails to scan, the dealer will re-scan the card to display the result to the Player.
Any failure of the Player's equipment including but not limited to network connection or computer problems, will not void the game result. The Player can check their bet history on the Report section.
Any Operator system malfunction and/or hardware failure during Live Dealer Dragon Tiger will void the play, and the particular table will be closed.
"""

In [9]:
llm = LLM()

RuntimeError: Failed to import transformers.integrations.bitsandbytes because of the following error (look up to see its traceback):
[WinError 193] %1 is not a valid Win32 application

In [ ]:

prompt = PROMPT.format(content=content)
answer = llm.ask(prompt)

print(answer)